Create a pair of Stellar wallets and make a transaction between them in testnet

In [1]:
# stellar-sdk >= 2.0.0 required
# create a completely new and unique pair of keys
# see more about KeyPair objects: https://stellar-sdk.readthedocs.io/en/latest/api.html#keypair
from stellar_sdk import Keypair

pair = Keypair.random()
print(f"Secret: {pair.secret}")
print(f"Public Key: {pair.public_key}")


Secret: SDKRM7G7PSK3ZCHMFA4LM343DJYX7RQTA5K7U7TC4DQ4SIX7UPLQMID2
Public Key: GAJH3MTLFLG52NHZUBPS7BGCBZPYKN4DEXYWZ2J5ASOCLVMXTASGUG5B


In [2]:
# The SDK does not have tools for creating test accounts, so you'll have to
# make your own HTTP request.

# if you're trying this on Python, install the `requests` library.
import requests
public_key = pair.public_key

#public_key = "GBFHY475WERNIEBU6VWOQ6B6LYCAVIH3AOHGR6XJQC3OW6PYTR5RCQHS"
response = requests.get(f"https://friendbot.stellar.org?addr={public_key}")
if response.status_code == 200:
    print(f"SUCCESS! You have a new account :)\n{response.text}")
else:
    print(f"ERROR! Response: \n{response.text}")

ERROR! Response: 
{
  "type": "https://stellar.org/horizon-errors/bad_request",
  "title": "Bad Request",
  "status": 400,
  "detail": "createAccountAlreadyExist (AAAAAAAAAGT/////AAAAAQAAAAAAAAAA/////AAAAAA=)"
}


In [3]:
from stellar_sdk import Server

server = Server("https://horizon-testnet.stellar.org")
public_key = "GBFHY475WERNIEBU6VWOQ6B6LYCAVIH3AOHGR6XJQC3OW6PYTR5RCQHS"
account = server.accounts().account_id(public_key).call()
for balance in account['balances']:
    print(f"Type: {balance['asset_type']}, Balance: {balance['balance']}")

Type: native, Balance: 10000.0000000


Create second account 

In [4]:
pair = Keypair.random()
print(f"Secret: {pair.secret}")
print(f"Public Key: {pair.public_key}")

Secret: SDPAILELWKWFHMNYG27U5C7FRE5N5AT3T4NSBIHG3H4TJ2TJINCI6QTA
Public Key: GDMNSDEK4TLML5Z6HGQXBP4HVD4BSTKRBAAXEH2DOI7YS76UMWZSR3SS


In [5]:

public_key = "GDMNSDEK4TLML5Z6HGQXBP4HVD4BSTKRBAAXEH2DOI7YS76UMWZSR3SS"
response = requests.get(f"https://friendbot.stellar.org?addr={public_key}")
if response.status_code == 200:
    print(f"SUCCESS! You have a new account :)\n{response.text}")
else:
    print(f"ERROR! Response: \n{response.text}")

SUCCESS! You have a new account :)
{
  "_links": {
    "self": {
      "href": "https://horizon-testnet.stellar.org/transactions/1dba2bece607d5b255b3d75346bd299cac566b2e50913388bf94c98e2ae6bb5f"
    },
    "account": {
      "href": "https://horizon-testnet.stellar.org/accounts/GBR7WTBMGNQUP6VEM2CJNDTVX54C33YTYE7UDKQKUNDEWRYSG4DXDIAT"
    },
    "ledger": {
      "href": "https://horizon-testnet.stellar.org/ledgers/65488"
    },
    "operations": {
      "href": "https://horizon-testnet.stellar.org/transactions/1dba2bece607d5b255b3d75346bd299cac566b2e50913388bf94c98e2ae6bb5f/operations{?cursor,limit,order}",
      "templated": true
    },
    "effects": {
      "href": "https://horizon-testnet.stellar.org/transactions/1dba2bece607d5b255b3d75346bd299cac566b2e50913388bf94c98e2ae6bb5f/effects{?cursor,limit,order}",
      "templated": true
    },
    "precedes": {
      "href": "https://horizon-testnet.stellar.org/transactions?order=asc\u0026cursor=281268818284544"
    },
    "succeeds": {

In [6]:

server = Server("https://horizon-testnet.stellar.org")
public_key = "GDMNSDEK4TLML5Z6HGQXBP4HVD4BSTKRBAAXEH2DOI7YS76UMWZSR3SS"
account = server.accounts().account_id(public_key).call()
for balance in account['balances']:
    print(f"Type: {balance['asset_type']}, Balance: {balance['balance']}")

Type: native, Balance: 10000.0000000


Sending and receiving coins

In [8]:
from stellar_sdk import Asset, Keypair, Network, Server, TransactionBuilder
from stellar_sdk.exceptions import NotFoundError, BadResponseError, BadRequestError

server = Server("https://horizon-testnet.stellar.org")
source_key = Keypair.from_secret("SDUIN6GBHQSGKTDGYPRPQHILJW6QTITNZOTM6KUOIKMCO25ZA2FNUJKV")
destination_id = "GDMNSDEK4TLML5Z6HGQXBP4HVD4BSTKRBAAXEH2DOI7YS76UMWZSR3SS"


# First, check to make sure that the destination account exists.
# You could skip this, but if the account does not exist, you will be charged
# the transaction fee when the transaction fails.
try:
    server.load_account(destination_id)
except NotFoundError:
    # If the account is not found, surface an error message for logging.
    raise Exception("The destination account does not exist!")

# If there was no error, load up-to-date information on your account.
source_account = server.load_account(source_key.public_key)

# Let's fetch base_fee from network
base_fee = server.fetch_base_fee()

In [10]:

# Start building the transaction.
transaction = (
    TransactionBuilder(
        source_account=source_account,
        network_passphrase=Network.TESTNET_NETWORK_PASSPHRASE,
        base_fee=base_fee,
    )
        # Because Stellar allows transaction in many currencies, you must specify the asset type.
        # Here we are sending Lumens.
        .append_payment_op(destination=destination_id, asset=Asset.native(), amount="10")
        # A memo allows you to add your own metadata to a transaction. It's
        # optional and does not affect how Stellar treats the transaction.
        .add_text_memo("Test Transaction")
        # Wait a maximum of three minutes for the transaction
        .set_timeout(10)
        .build()
)

# Sign the transaction to prove you are actually the person sending it.
transaction.sign(source_key)

try:
    # And finally, send it off to Stellar!
    response = server.submit_transaction(transaction)
    print(f"Response: {response}")
except (BadRequestError, BadResponseError) as err:
    print(f"Something went wrong!\n{err}")

Response: {'memo': 'Test Transaction', 'memo_bytes': 'VGVzdCBUcmFuc2FjdGlvbg==', '_links': {'self': {'href': 'https://horizon-testnet.stellar.org/transactions/a2ac084af8b3c0574a78211710e5ab1344da9999d14fe4b4c0b2ec0b68dbae32'}, 'account': {'href': 'https://horizon-testnet.stellar.org/accounts/GBFHY475WERNIEBU6VWOQ6B6LYCAVIH3AOHGR6XJQC3OW6PYTR5RCQHS'}, 'ledger': {'href': 'https://horizon-testnet.stellar.org/ledgers/65669'}, 'operations': {'href': 'https://horizon-testnet.stellar.org/transactions/a2ac084af8b3c0574a78211710e5ab1344da9999d14fe4b4c0b2ec0b68dbae32/operations{?cursor,limit,order}', 'templated': True}, 'effects': {'href': 'https://horizon-testnet.stellar.org/transactions/a2ac084af8b3c0574a78211710e5ab1344da9999d14fe4b4c0b2ec0b68dbae32/effects{?cursor,limit,order}', 'templated': True}, 'precedes': {'href': 'https://horizon-testnet.stellar.org/transactions?order=asc&cursor=282046207365120'}, 'succeeds': {'href': 'https://horizon-testnet.stellar.org/transactions?order=desc&cursor=

In [1]:
import requests
from stellar_sdk import Keypair

def create_new_stellar_account():
    pair = Keypair.random()
    #print(f"Secret: {pair.secret}")
    print(f"Stellar public Key: {pair.public_key}")
    return pair

def activate_new_stellar_account(public_key):
    
    response = requests.get(f"https://friendbot.stellar.org?addr={public_key}")
    if response.status_code == 200:
        print(f"SUCCESS! You have a new account :)\n{response.text}")
    else:
        print(f"ERROR! Response: \n{response.text}")
    return response.text


In [7]:
stellar_wallet = create_new_stellar_account()

Stellar public Key: GCFGWTTYNFTO3YAIV6LHQ6N5SVYCPSJZFFT7QLSORDMWV2UBQV6ACISW


In [11]:
stellar_wallet.secret

'SCODQKESWRVYS45G4DWHIE4DYOMLURUHOWW4E5X524WCDUYM2WQQKHTV'

In [9]:
activate_new_stellar = activate_new_stellar_account(stellar_wallet.public_key)


SUCCESS! You have a new account :)
{
  "_links": {
    "self": {
      "href": "https://horizon-testnet.stellar.org/transactions/f9fdf74548074f89000c5fb2e0771ba4835f3c6c5261603f9946a0f7b2e7dd04"
    },
    "account": {
      "href": "https://horizon-testnet.stellar.org/accounts/GA75DETSR3HZEQDPAZVAUS6N5KVZDJP34JN437HS42VWRK2N3VRD67UI"
    },
    "ledger": {
      "href": "https://horizon-testnet.stellar.org/ledgers/79418"
    },
    "operations": {
      "href": "https://horizon-testnet.stellar.org/transactions/f9fdf74548074f89000c5fb2e0771ba4835f3c6c5261603f9946a0f7b2e7dd04/operations{?cursor,limit,order}",
      "templated": true
    },
    "effects": {
      "href": "https://horizon-testnet.stellar.org/transactions/f9fdf74548074f89000c5fb2e0771ba4835f3c6c5261603f9946a0f7b2e7dd04/effects{?cursor,limit,order}",
      "templated": true
    },
    "precedes": {
      "href": "https://horizon-testnet.stellar.org/transactions?order=asc\u0026cursor=341097712721920"
    },
    "succeeds": {

In [10]:
activate_new_stellar

'{\n  "_links": {\n    "self": {\n      "href": "https://horizon-testnet.stellar.org/transactions/f9fdf74548074f89000c5fb2e0771ba4835f3c6c5261603f9946a0f7b2e7dd04"\n    },\n    "account": {\n      "href": "https://horizon-testnet.stellar.org/accounts/GA75DETSR3HZEQDPAZVAUS6N5KVZDJP34JN437HS42VWRK2N3VRD67UI"\n    },\n    "ledger": {\n      "href": "https://horizon-testnet.stellar.org/ledgers/79418"\n    },\n    "operations": {\n      "href": "https://horizon-testnet.stellar.org/transactions/f9fdf74548074f89000c5fb2e0771ba4835f3c6c5261603f9946a0f7b2e7dd04/operations{?cursor,limit,order}",\n      "templated": true\n    },\n    "effects": {\n      "href": "https://horizon-testnet.stellar.org/transactions/f9fdf74548074f89000c5fb2e0771ba4835f3c6c5261603f9946a0f7b2e7dd04/effects{?cursor,limit,order}",\n      "templated": true\n    },\n    "precedes": {\n      "href": "https://horizon-testnet.stellar.org/transactions?order=asc\\u0026cursor=341097712721920"\n    },\n    "succeeds": {\n      "hr